In [16]:
"""
Author: Zaw
Findin the three most deviations of Predicting the LogP of molecules by using svm
"""
import numpy as np
import pandas as pd
import math
import time
import os
import sys
import warnings
warnings.simplefilter(action='ignore')
from sklearn import svm
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from hyperopt import fmin,tpe,hp,STATUS_OK, Trials
import pickle
start = time.time()

In [17]:
# Data path
path = os.getcwd()
dirname = os.path.dirname(path)
data_path = dirname+"/LogP/data/extract_data.csv"
col_list=['LogP','Exp_RT']
lc_df = pd.read_csv(data_path,usecols=col_list)
col_smile_list=['SMILES','LogP','Exp_RT']
lc_smiles_df = pd.read_csv(data_path,usecols=col_smile_list)

# Remove non_retained molecules
# index=lc_df[lc_df['Exp_RT'] < 180].index
# lc_df.drop(lc_df[lc_df['Exp_RT'] < 180].index,inplace=True)

# Import descriptor file
path = os.getcwd()
dirname = os.path.dirname(path)
data_path = dirname+"/LogP/data/descriptors.csv"
des_df = pd.read_csv(data_path,index_col=0)

# Remove non_retained molecules
#des_df  = des_df.drop(des_df.index[[index]])
#lc_smiles_df = lc_smiles_df.drop(lc_smiles_df[[index]])
# combine two dataframe
data_comb = pd.concat([lc_smiles_df,des_df],axis=1)

In [18]:
data_comb

,SMILES,LogP,Exp_RT,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,CCSC(=N)N,0.67,93.5,6.635046,0.206019,6.635046,0.206019,0.378489,104.178,96.114,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,COC(=O)N1CCN(C(=O)Cc2ccc(Cl)c(Cl)c2)[C@H](CN2C...,2.64,687.8,12.990628,-0.335181,12.990628,0.035905,0.760602,414.333,389.133,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Oc1cccc2c(O)nccc12,1.73,583.6,9.356759,-0.052407,9.356759,0.052407,0.617197,161.160,154.104,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,NCCc1ccc(S(=O)(=O)F)cc1,1.05,653.1,12.386283,-4.565776,12.386283,0.311786,0.740582,203.238,193.158,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,CNC(=O)c1cc2c(cn1)[nH]c1ccccc12,1.33,705.6,11.531255,-0.169277,11.531255,0.169277,0.665746,225.251,214.163,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2066,OCCn1cc(-c2ccc3c(c2)CC/C3=N\O)c(-c2ccncc2)n1,2.05,603.4,9.236427,0.039104,9.236427,0.039104,0.567510,334.379,316.235,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2067,CC(C)OC(=O)N1CCC(Oc2ncnc3c2cnn3-c2ccc(S(C)(=O)...,1.59,1061.4,14.683496,-3.539697,14.683496,0.054651,0.550748,477.518,453.326,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2068,CCC(=O)O[C@]1(C(=O)CO)CC[C@H]2[C@@H]3CCC4=CC(=...,3.73,1059.3,12.960935,-1.190444,12.960935,0.073898,0.724147,402.531,368.259,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2069,O=C(c1cc2ccccc2[nH]1)N1CCCC1,1.70,794.3,12.111258,0.133056,12.111258,0.133056,0.777073,214.268,200.156,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
# # drop the smile column which can not be used for training
# data_comb = data_comb.drop(['SMILES'],axis=1)

# # Filling the nan with mean values in des_with_lc
# for col in data_comb:
#         data_comb[col].fillna(data_comb[col].mean(),inplace=True)
# # Remove features with low Variance(threshold <= 0.05)

# data_var = data_comb.var()
# del_feat = list(data_var[data_var <= 0.05].index)
# data_comb.drop(columns=del_feat, inplace=True)

# # Remove features with correlation(threshold >0.95)
# corr_matrix = data_comb.corr().abs()
# mask = np.triu(np.ones_like(corr_matrix,dtype=bool))
# tri_df = corr_matrix.mask(mask)
# to_drop =  [ c for c in tri_df.columns if any(tri_df[c] > 0.95)]
# data = data_comb.drop(to_drop,axis=1)

In [20]:
# The dataset with lc informations
#load the model with lc
path = os.getcwd()
dirname = os.path.dirname(path)

# load the model
file_path = dirname+"/logP/hyperparameters/svm_lc.pkl"
best_parameters_with_lc = pickle.load(open(file_path,'rb'))

In [21]:
#Single random run with RT
data_set_1 = des_df
data_set_2 = lc_df
des_with_lc = pd.concat([data_set_1,data_set_2],axis=1)
des_with_lc_feat_corr = des_with_lc.columns[des_with_lc.corrwith(des_with_lc['LogP']) >=0.90][:-1]
des_with_lc = des_with_lc.drop(columns=des_with_lc_feat_corr)

# Rescale the feature
def scale(col):
    return (col - np.min(col))/(np.max(col)-np.min(col))

# Filling the nan with mean values in des_with_lc
for col in des_with_lc:
    des_with_lc[col].fillna(des_with_lc[col].mean(),inplace=True)

# Remove columns with zero vlues
des_with_lc = des_with_lc.loc[:,(des_with_lc**2).sum() != 0]
data = des_with_lc.drop(['LogP'],axis=1)

# Remove features with low Variance(threshold<=0.05)
data_var = data.var()
del_feat = list(data_var[data_var <= 0.05].index)
data.drop(columns=del_feat, inplace=True)

# Remove features with correlation(threshold > 0.95)
corr_matrix = data.corr().abs()
mask = np.triu(np.ones_like(corr_matrix,dtype=bool))
tri_df = corr_matrix.mask(mask)
to_drop =  [ c for c in tri_df.columns if any(tri_df[c] > 0.95)]
data = data.drop(to_drop,axis=1)

# Scale the features
cols = list(data)
data[cols] = data[cols].apply(scale,axis=0)
data = pd.concat([data,des_with_lc['LogP']],axis=1)

# Single random run

# data set preparation
train , test = train_test_split(data,train_size = 0.5,shuffle=True)
#validate , test = train_test_split(rest, train_size = 0.5, shuffle=True)

# training set
data_tra_x = train.drop(['LogP'],axis=1)
data_tra_y = train['LogP']

# # validation set
# data_val_x = validate.drop(['LogP'],axis=1)
# data_val_y = validate['LogP']

# test set
data_tes_x = test.drop(['LogP'],axis=1)
data_tes_y = test['LogP']

best_model = svm.SVR(kernel='linear',cache_size=2000,max_iter=10000, C = best_parameters_with_lc['C'])

best_model.fit(data_tra_x,data_tra_y)

# training error
tra_pred = best_model.predict(data_tra_x)
tra_results = ['tra',mean_squared_error(data_tra_y,tra_pred),
            mean_squared_error(data_tra_y,tra_pred,squared=False),
            mean_absolute_error(data_tra_y,tra_pred),
            r2_score(data_tra_y,tra_pred)]

# # validation error
# val_pred = best_model.predict(data_val_x)
# val_results = ['val',mean_squared_error(data_val_y,val_pred),
#             mean_squared_error(data_val_y,val_pred,squared=False),
#             mean_absolute_error(data_val_y,val_pred),
#             r2_score(data_val_y,val_pred)]
# testing error
tes_pred = best_model.predict(data_tes_x)
tes_results = ['tes',mean_squared_error(data_tes_y,tes_pred),
            mean_squared_error(data_tes_y,tes_pred,squared=False),
            mean_absolute_error(data_tes_y,tes_pred),
            r2_score(data_tes_y,tes_pred)]


all_set = tra_results,tes_results


all_set_df = pd.DataFrame(all_set,columns=['set','mse','rmse','mae','r2'])
print(all_set_df)
#all_set_df.to_excel(dirname+"/logP/results/mlp_single_rt.xlsx")
# print('\nSingle random run with LC information is done.')

# run_best_model(des_df,lc_df)

   set       mse      rmse       mae        r2
0  tra  0.259037  0.508957  0.342604  0.904083
1  tes  0.305863  0.553049  0.381150  0.897664


In [22]:
# Predictions
predictions = pd.DataFrame(tes_pred,columns=['Predicted LogP'])

data_test = test
data_test['index'] = data_test.index

data_comb_smiles = data_comb
data_comb_smiles['index'] = data_comb_smiles.index

result = pd.merge(data_test['index'],data_comb_smiles)

test_df = result[['SMILES','LogP']]

In [23]:
test_df

,SMILES,LogP
0,Cn1cc(C=O)c2nccnc21,0.33
1,CN1C(=C=Cc2cc(I)c(O)c(I)c2)C(C)(C)c2ccccc21,6.48
2,O=C(Nc1nccs1)c1ccc2c(c1)CCCC2,3.86
3,O=C1C=C2C=C[C@@H]3C[C@@]2(O1)[C@H]1CCCCN31,1.41
4,CC1(C)CCC[C@@](C)([C@H]2CC[C@]3(C)[C@@H]2[C@H]...,4.32
...,...,...
1031,COc1ccc2c(c1)CCc1sc(NC(=O)c3ccc(Br)cc3)nc1-2,5.75
1032,Cc1ccc(-c2nc3cccc(C)n3c2Nc2ccc3c(c2)OCCO3)cc1,4.55
1033,O=C(NCc1ccccn1)C(=O)Nc1nccs1,0.52
1034,Cn1cnc([N+](=O)[O-])c1S(=O)(=O)Oc1cccc2ccccc12,2.91


In [24]:
# create dataframe with predicted LogP and Experimenatal logP
predictions = pd.DataFrame(tes_pred,columns=['Predicted LogP'])

svm_df = pd.concat([test_df,predictions],axis=1)

svm_df.rename(columns={'LogP':'Experimental LogP'},inplace=True)

svm_df['Predicted LogP'] = svm_df['Predicted LogP'].round(decimals=2)


In [25]:
svm_df

,SMILES,Experimental LogP,Predicted LogP
0,Cn1cc(C=O)c2nccnc21,0.33,0.47
1,CN1C(=C=Cc2cc(I)c(O)c(I)c2)C(C)(C)c2ccccc21,6.48,5.85
2,O=C(Nc1nccs1)c1ccc2c(c1)CCCC2,3.86,3.65
3,O=C1C=C2C=C[C@@H]3C[C@@]2(O1)[C@H]1CCCCN31,1.41,0.65
4,CC1(C)CCC[C@@](C)([C@H]2CC[C@]3(C)[C@@H]2[C@H]...,4.32,4.90
...,...,...,...
1031,COc1ccc2c(c1)CCc1sc(NC(=O)c3ccc(Br)cc3)nc1-2,5.75,5.52
1032,Cc1ccc(-c2nc3cccc(C)n3c2Nc2ccc3c(c2)OCCO3)cc1,4.55,4.78
1033,O=C(NCc1ccccn1)C(=O)Nc1nccs1,0.52,0.54
1034,Cn1cnc([N+](=O)[O-])c1S(=O)(=O)Oc1cccc2ccccc12,2.91,2.17


In [31]:
# Find deviation
svm_df['Deviation'] = (svm_df['Experimental LogP']- svm_df['Predicted LogP'])

In [32]:
# round off decimal value
svm_df['Deviation'] = svm_df['Deviation'].round(decimals=2)

In [33]:
svm_df

,SMILES,Experimental LogP,Predicted LogP,Deviation
0,Cn1cc(C=O)c2nccnc21,0.33,0.47,-0.14
1,CN1C(=C=Cc2cc(I)c(O)c(I)c2)C(C)(C)c2ccccc21,6.48,5.85,0.63
2,O=C(Nc1nccs1)c1ccc2c(c1)CCCC2,3.86,3.65,0.21
3,O=C1C=C2C=C[C@@H]3C[C@@]2(O1)[C@H]1CCCCN31,1.41,0.65,0.76
4,CC1(C)CCC[C@@](C)([C@H]2CC[C@]3(C)[C@@H]2[C@H]...,4.32,4.90,-0.58
...,...,...,...,...
1031,COc1ccc2c(c1)CCc1sc(NC(=O)c3ccc(Br)cc3)nc1-2,5.75,5.52,0.23
1032,Cc1ccc(-c2nc3cccc(C)n3c2Nc2ccc3c(c2)OCCO3)cc1,4.55,4.78,-0.23
1033,O=C(NCc1ccccn1)C(=O)Nc1nccs1,0.52,0.54,-0.02
1034,Cn1cnc([N+](=O)[O-])c1S(=O)(=O)Oc1cccc2ccccc12,2.91,2.17,0.74


In [36]:
# dataframe to csv
svm_df.to_csv('svm_exp_pre.csv')

In [37]:
# The three most deviation values
svm_df.iloc[list(svm_df['Deviation'].nlargest(3).index)]

,SMILES,Experimental LogP,Predicted LogP,Deviation
159,CN(CO)c1nc(N(C)CO)nc(N(C)CO)n1,0.48,-1.85,2.33
448,CC(C)(C)C(=O)OCOP(=O)(COCCn1cnc2c(N)ncnc21)OCO...,3.06,0.88,2.18
621,c1cc(-c2nn[nH]n2)ccc1-c1nn[nH]n1,1.94,0.30,1.64


In [39]:
svm_df[svm_df['Deviation']==2.18]['SMILES'].item()

'CC(C)(C)C(=O)OCOP(=O)(COCCn1cnc2c(N)ncnc21)OCOC(=O)C(C)(C)C'

In [38]:
import pubchempy

smiles ='CN(CO)c1nc(N(C)CO)nc(N(C)CO)n1'
compounds = pubchempy.get_compounds(smiles, namespace='smiles')
match = compounds[0]
print(match.iupac_name)

[[4,6-bis[hydroxymethyl(methyl)amino]-1,3,5-triazin-2-yl]-methylamino]methanol


In [40]:
smiles ='CC(C)(C)C(=O)OCOP(=O)(COCCn1cnc2c(N)ncnc21)OCOC(=O)C(C)(C)C'
compounds = pubchempy.get_compounds(smiles, namespace='smiles')
match = compounds[0]
print(match.iupac_name)

[2-(6-aminopurin-9-yl)ethoxymethyl-(2,2-dimethylpropanoyloxymethoxy)phosphoryl]oxymethyl 2,2-dimethylpropanoate


In [41]:
smiles ='c1cc(-c2nn[nH]n2)ccc1-c1nn[nH]n1'
compounds = pubchempy.get_compounds(smiles, namespace='smiles')
match = compounds[0]
print(match.iupac_name)

5-[4-(2H-tetrazol-5-yl)phenyl]-2H-tetrazole
